In [1]:
pwd

'/home/labs/testing/class20/exemplar/Deep-Exemplar-based-Video-Colorization'

In [3]:
import torch

In [5]:
torch.cuda.is_available()

True

In [8]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%run train.py --data_root='../train_data/data_root' --epoch=1 --gpu_ids="0"

------------ Options -------------
batch_size: 16
beta1: 0.5
checkpoint_dir: checkpoints/default
contextual_loss_direction: forward
data_root: ../train_data/data_root
data_root_imagenet: ../train_data/data_root/lioness/reference
domain_invariant: False
epoch: 40
gpu_ids: 0
ic: 7
image_size: [216, 384]
load_pretrained_model: True
lr: 1e-05
lr_gamma: 0.1
lr_step: 100
luminance_noise: 2.0
nonzero_placeholder_probability: 0.0
permute_data: True
print_step: 2
real_reference_probability: 0.7
resume: False
resume_epoch: 0
tb_log_step: 50
weight_consistent: 0.02
weight_contextual: 0.2
weight_gan: 0.2
weight_nonlocal_consistent: 0.0
weight_nonlocal_smoothness: 0.0
weight_perceptual: 0.001
weight_smoothness: 5.0
weigth_l1: 2.0
with_bad: True
with_mid: True
workers: 8
-------------- End ----------------
running on GPU [0]
initializing dataloader
##### parsing image pairs in ../train_data/data_root: 25 pairs #####
Dana Hamalca
replace all deconv with [nearest + conv]
replace all batchnorm with ins

AttributeError: module '__main__' has no attribute '__spec__'

In [6]:
ls

assets/       LICENSE*    requirements.txt*  test.py*
checkpoints/  models/     runs/              train.py*
data/         opt.txt*    sample_videos/     try_run_train.ipynb*
lib/          README.md*  tensorboardX/      utils/


In [67]:
""" get a path to MASK directory and extract only the relevant masks & rename them """

import os
import shutil


def get_relevant_frames(path, sframe=0, eframe=25, output_path = "./output"):
    relevant_flow = []

    # forward
    for i in range(sframe, eframe):
        relevant_flow.append((i, i+1))

    # backward
#     for i in range(eframe, sframe, -1):
#         relevant_flow.append((i, i-1))

    # assume the images' names are of the format: 'mask_000010_000012.png'
    images = [( f, (int(f[5:11]), int(f[12:18])) ) for f in os.listdir(path)]  # [('mask_000031_000033.png', (31, 33)), ...]

    if not os.path.isdir(output_path):
        os.mkdir(output_path)

    for im in images:
        if im[1] in relevant_flow:
            # move this file to another directory
            shutil.copyfile(os.path.join(path, im[0]), os.path.join(output_path, f'{im[0][5:11]}_mask.pgm'))

get_relevant_frames('../flow_mask', 0, 25, output_path='../train_data/data_root/lioness/mask')



In [58]:
""" create FLOW directory: extract only the relevant flows & rename them """

import os
import shutil


def get_relevant_frames_flow(path, sframe=0, eframe=25, output_path = "./output"):
    relevant_flow = []

    # forward
    for i in range(sframe, eframe):
        relevant_flow.append((i, i+1))

    # backward
    for i in range(eframe, sframe, -1):
        relevant_flow.append((i, i-1))

    # assume the images' names are of the format: 'mask_000010_000012.png'
    images = [( f, (int(f[5:11]), int(f[12:18])) ) for f in os.listdir(path)]  # [('mask_000031_000033.png', (31, 33)), ...]

    if not os.path.isdir(output_path):
        os.mkdir(output_path)

    for im in images:
        if im[1] in relevant_flow:
            # move this file to another directory & change name
            file = im[0]
            
            _, im1, im2 = file.split(".")[0].split('_')
            if int(im1) < int(im2):
                name = f"{im1}_forward.raw"
            else:
                name = f"{im1}_backward.raw"
            shutil.copyfile(os.path.join(path, file), os.path.join(output_path, name))


get_relevant_frames_flow('../flow', 0, 25, output_path='../train_data/data_root/flow')


In [61]:
""" rename color frame names and move to another dir """

import os
import shutil


def move_rename_color_frames(path, sframe=0, eframe=25, output_path = "./output"):
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
        
    for f in os.listdir(path):
        name = f.split('_')[1]
        num = int(name.split('.')[0])
        if num >= sframe and num <= eframe:
            shutil.copyfile(os.path.join(path, f), os.path.join(output_path, name))
        
move_rename_color_frames('../color_down_png', 0, 25, output_path='../train_data/data_root/input_pad')


In [4]:
# Shira's Code - generates 'pairs_output_new.txt'
import os


def gen_pairs_output_new(basedir='data_root'):
    for fn in os.listdir(basedir):
        if not os.path.isdir(os.path.join(basedir, fn)):
            continue  # Not a directory
        frames_dir = os.path.join(basedir, fn, 'input_pad')
        frames = [f for f in os.listdir(frames_dir) if os.path.isfile(os.path.join(frames_dir, f))]
        file = os.path.join(basedir, fn, 'pairs_output_new.txt')
        frames.sort()
        with open(file, 'w') as f:
            for i in range(len(frames)-1):
                f.write(frames[i] + " ")
                f.write(frames[i+1] + " ")
                f.write("reference_video_name ")
                f.write("reference_video_name ")
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0] + ' ')
                f.write(frames[0])

                f.write("\n")

gen_pairs_output_new(basedir='../train_data/data_root')

In [2]:
""" create flow directory fron FlowNet2 outputs """


import os
from os.path import join
import shutil


def func(source_path, sframe=0, eframe=25, output_path = "./output"):
    
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    
    forward_path = join(source_path, 'lioness_flow_forward')
    backward_path = join(source_path, 'lioness_flow_back')
    
    # forward
    for f in os.listdir(forward_path):
        num = f.split('.')[0]
        if int(num) >= sframe and int(num) <= eframe:
            shutil.copyfile(join(forward_path, f), join(output_path, f'{num}_forward.flo'))
    
    # backward
    for f in os.listdir(backward_path):
        num = f.split('.')[0]
        if int(num) >= sframe and int(num) <= eframe:
            shutil.copyfile(join(forward_path, f), join(output_path, f'{num}_backward.flo'))
        

func("../FLowNet2_output", 0, 25, output_path='../train_data/data_root/lioness/flow')

# /home/labs/waic/shirawe/colorization/exemplar/FLowNet2_output/
# lioness_flow_forward
# lioness_flow_back
